# yolo を使ったobject detection



セッション切れ回避

```
function ClickConnect(){
  console.log("60sごとに再接続");
  document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,1000*60);
```

In [23]:
# config

CFG = {
    "fold_num": 5,
    "val_fold": 0,
    "bach_size": 32,
    "epochs": 50,
    "weights": "yolov5s.pt",
    "dim": 640
}
IS_TEST = True

In [2]:
#ColaboratoryかKaggleNotebookか判別
import sys
import os
from pathlib import Path

MAIN_INPUT_NAME = "kaggle-shopee"
MAIN_INPUT_ZIP = f'../input/{MAIN_INPUT_NAME}.zip'


if 'google.colab' in sys.modules:  # colab環境
    INPUT = Path('/content/input/')

    # drive mount
    from google.colab import drive
    drive.mount('/content/drive/')

    # working dir に移動
    os.chdir(f'/content/drive/MyDrive/git/{MAIN_INPUT_NAME}/working')
    print("cwd:", os.getcwd())

    # 必要な input を指定先へ unzip
    INPUT_SHOPEE = os.path.join(INPUT,"shopee-product-matching")
    os.makedirs(INPUT_SHOPEE, exist_ok=True)
    !unzip -q {"../input/shopee-product-matching.zip"} -d {INPUT_SHOPEE}
    !unzip -q {"../input/save.zip"} -d {INPUT}
    print("unzip INPUT")
    
elif 'kaggle_web_client' in sys.modules:  # kaggle環境
    INPUT = Path('../input/')


Mounted at /content/drive/
cwd: /content/drive/MyDrive/git/kaggle-shopee/working
unzip INPUT


In [6]:
import glob
import os
import pandas as pd
import yaml
import numpy as np
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
%matplotlib inline

In [16]:
!pip install GPUtil
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():                
    torch.cuda.empty_cache()
    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)
free_gpu_cache()


  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=8089395cdf6c9f3173c8fdb408dea459fe42e604693431c086d4379b0d2a3d95
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


In [30]:
# annotation 済み一覧を作る
anno_file_list = glob.glob(os.path.join(INPUT, "save/*_*.txt"))
anno_df = pd.DataFrame(anno_file_list, columns=["file_path"])
anno_df = pd.concat([anno_df, anno_df['file_path'].str.extract('.+/(\d+)_(.+)\.txt', expand=True)], axis=1).rename(columns={0: 'label_group', 1: 'image'})
anno_df.image = anno_df.image + ".jpg"
anno_df["image_path"] = INPUT_SHOPEE + "/train_images/" + anno_df.image
anno_df.head()

,file_path,label_group,image,image_path
0,/content/input/save/21546494_208dc3897f1de16fa...,21546494,208dc3897f1de16fa5b4377217c62d9f.jpg,/content/input/shopee-product-matching/train_i...
1,/content/input/save/71375986_11720be0a09ece2ef...,71375986,11720be0a09ece2efd922136d5ae0483.jpg,/content/input/shopee-product-matching/train_i...
2,/content/input/save/109922603_3c7acd1d467ac38f...,109922603,3c7acd1d467ac38f52ba8ede6d51a531.jpg,/content/input/shopee-product-matching/train_i...
3,/content/input/save/40040342_b0f97aacae490c5bf...,40040342,b0f97aacae490c5bf0f838a9ec4e7695.jpg,/content/input/shopee-product-matching/train_i...
4,/content/input/save/33999540_657db542bce7d3c41...,33999540,657db542bce7d3c41296d09eca9879f9.jpg,/content/input/shopee-product-matching/train_i...


In [31]:
# group kfold
gkf  = GroupKFold(n_splits = CFG["fold_num"])
anno_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(anno_df, groups = anno_df.label_group.tolist())):
    anno_df.loc[val_idx, 'fold'] = fold
anno_df.head()

,file_path,label_group,image,image_path,fold
0,/content/input/save/21546494_208dc3897f1de16fa...,21546494,208dc3897f1de16fa5b4377217c62d9f.jpg,/content/input/shopee-product-matching/train_i...,2
1,/content/input/save/71375986_11720be0a09ece2ef...,71375986,11720be0a09ece2efd922136d5ae0483.jpg,/content/input/shopee-product-matching/train_i...,2
2,/content/input/save/109922603_3c7acd1d467ac38f...,109922603,3c7acd1d467ac38f52ba8ede6d51a531.jpg,/content/input/shopee-product-matching/train_i...,2
3,/content/input/save/40040342_b0f97aacae490c5bf...,40040342,b0f97aacae490c5bf0f838a9ec4e7695.jpg,/content/input/shopee-product-matching/train_i...,3
4,/content/input/save/33999540_657db542bce7d3c41...,33999540,657db542bce7d3c41296d09eca9879f9.jpg,/content/input/shopee-product-matching/train_i...,4


# yolo

In [35]:
# image を一旦コピー
images_dir = '/content/yolo_data/images'
os.makedirs(images_dir, exist_ok = True)
for file in tqdm(anno_df.image_path):
    shutil.copy(file, images_dir)

# labels をコピー
labels_dir = '/content/yolo_data/labels'
os.makedirs(labels_dir, exist_ok = True)
for file in tqdm(anno_df.file_path):
    new_name = file.split("_")[1]
    shutil.copy(file, os.path.join(labels_dir, new_name))


In [21]:
# fold ごとに生成
for fold in range(CFG["fold_num"]):
    print(f"fold: {fold}")
    # image
    train_images = []
    val_images   = []
    val_images += list(images_dir+'/'+anno_df[anno_df.fold==fold].image)
    train_images += list(images_dir+'/'+anno_df[anno_df.fold!=fold].image)
    """
    # anno (yolo はパスを見て images と labels を切り替える)
    train_anno = []
    val_anno   = []
    val_anno += list(anno_df[anno_df.fold==fold].file_path)
    train_anno += list(anno_df[anno_df.fold!=fold].file_path)
    """

    # fold ごとに使うデータのパスを記録
    train_txt_file = f'/content/yolo_data/train_{fold}.txt'
    val_txt_file = f'/content/yolo_data/val_{fold}.txt'
    classes = [0]
    with open(train_txt_file, 'w') as f:
        for path in train_images:
            f.write(path+'\n')
                
    with open(val_txt_file, 'w') as f:
        for path in val_images:
            f.write(path+'\n')

    # yaml for setting
    yaml_file = f"/content/yolo_data/fold_{fold}.yaml"
    data = dict(
        train =  train_txt_file ,
        val   =  val_txt_file,
        nc    = 1,
        names = classes
        )
    with open(yaml_file, 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)
    f = open(yaml_file, 'r')
    print('\nyaml:')
    print(f.read())
    print()

fold: 0

yaml:
names:
- 0
nc: 1
train: /content/yolo_data/train_0.txt
val: /content/yolo_data/val_0.txt


fold: 1

yaml:
names:
- 0
nc: 1
train: /content/yolo_data/train_1.txt
val: /content/yolo_data/val_1.txt


fold: 2

yaml:
names:
- 0
nc: 1
train: /content/yolo_data/train_2.txt
val: /content/yolo_data/val_2.txt


fold: 3

yaml:
names:
- 0
nc: 1
train: /content/yolo_data/train_3.txt
val: /content/yolo_data/val_3.txt


fold: 4

yaml:
names:
- 0
nc: 1
train: /content/yolo_data/train_4.txt
val: /content/yolo_data/val_4.txt




In [14]:
#!git clone https://github.com/ultralytics/yolov5  # clone repo
os.chdir("yolov5")
!pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 5931, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 5931 (delta 90), reused 83 (delta 41), pack-reused 5770
Receiving objects: 100% (5931/5931), 8.39 MiB | 9.48 MiB/s, done.
Resolving deltas: 100% (4033/4033), done.
     |████████████████████████████████| 645kB 4.3MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [36]:
for fold in range(CFG["fold_num"]):
    if fold==1 and IS_TEST==True:
        break
    exp_name = f'exp_fold{fold}'
    data_file = f"/content/yolo_data/fold_{fold}.yaml"

    epochs = CFG["epochs"]
    bach_size = CFG["bach_size"]
    weights  = CFG["weights"]
    dim = CFG["dim"]
    
    free_gpu_cache()
    
    # 前回の実験を削除
    !rm -rf ./runs/train/{exp_name}

    !python train.py \
     --epochs $epochs \
     --batch-size $bach_size \
     --data $data_file \
     --weights $weights \
     --img $dim \
     --name $exp_name

github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 2021-4-24 torch 1.8.1+cu101 CUDA:0 (Tesla V100-SXM2-16GB, 16160.5MB)

Namespace(adam=False, artifact_alias='latest', batch_size=32, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/yolo_data/fold_0.yaml', device='', entity=None, epochs=50, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp_fold0', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp_fold0', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=32, upload_dataset=False, weights='yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-04-24 00:29:13.381550: I tensorflow/stream_executor/platform/default/dso_loader.cc